In [ ]:
# Imports

import numpy as np
import pandas as pd

import asyncio

from pymepix.channel.channel_types import ChannelDataType, Commands

from collections import deque

import time
import holoviews as hv
from holoviews import opts
from holoviews import streams
from holoviews.streams import Pipe
from bokeh.plotting import show
hv.extension('bokeh')

import sys
import os

%run stream_timepix_utils.py

In [ ]:
'''A callback function declaration is:
    
@callback
def on_data(in_data):
    print(in_data['type'])
    
Stop sending data to callback functions:

unregister_all()
    
'''
unregister_all()

In [ ]:
"""Receiving the pixel data and plotting it"""

H_VMI = np.zeros((255, 255))
bins_VMI = (range(0, 256), range(0, 256))

pixel_df_data = pd.DataFrame({'x': [], 'y': [], 'toa': [], 'tot': [],})
pixel_buffer = Pipe(pixel_df_data)

def my_VMI(data):
    global H_VMI, bins_VMI
    
    H_VMI += np.histogram2d(data['x'], data['y'], bins=bins_VMI)[0]
    #H_VMI += np.histogram2d(np.random.randint(255, size=100), np.random.randint(255, size=100),\
    #                       bins=bins_VMI)[0]
    
    #H_VMI_bg += np.histogram2d(df_bg['x'], df_bg['y'], bins=bins_VMI)[0]
    return hv.Image(H_VMI.T[::-1], bounds=[bins_VMI[0][0], bins_VMI[1][0], bins_VMI[0][-1], bins_VMI[1][-1]], label='All').opts(logz=True, clim=(1e-3, None), cmap="jet", frame_height=400, frame_width=400)

@callback
def on_data_pixel(in_data):
    global pixel_buffer
    if in_data['type'] == ChannelDataType.PIXEL.value:
        '''recieved pixel data'''
        #print(type(in_data['data']))
        
        print(in_data)
        
        pixel_buffer.send(in_data['data'])        


In [ ]:
hv.DynamicMap(my_VMI, streams=[pixel_buffer]) 

In [ ]:
"""Receiving the ToF data during recording (from scans) and accumulating it in records dictionary """

records = {}
in_record = False
records_accu = pd.DataFrame()
record_num = 0

@callback
def on_data_command(in_data):
    global in_record
    global records_accu
    global record_num
    global records
    
    if in_data['type'] == ChannelDataType.COMMAND.value:
        '''received COMMAND data'''
        if in_data['data'] == Commands.START_RECORD.value:
            in_record = True
            records_accu = pd.DataFrame()
            record_num += 1
        elif in_data['data'] == Commands.STOP_RECORD.value:
            in_record = False
            records[record_num] = records_accu
            
    elif in_data['type'] == ChannelDataType.TOF.value and in_record == True:
        '''TOF data'''
        #print('added to accu')
        records_accu = pd.concat([records_accu, in_data['data']])
    
        

In [ ]:
# Plotting data (simple averaging) from records dictionary in upper cell



In [ ]:
"""Plotting pixel (y) vs ToF"""

tof_df_data = pd.DataFrame({'nr':[], 'x': [], 'y': [], 'tof': [], 'tot': [],})
tof_buffer = Pipe(tof_df_data)

bins_y_tof = (np.linspace(0, 10, 1000), range(256))
H_y_tof = np.zeros((len(bins_y_tof[0])-1, len(bins_y_tof[1])-1))

def my_x_tof(data):
    global H_y_tof, bins_y_tof
    
    #print(data)    
    #H_y_tof += np.histogram2d(data['tof'], data['y'], bins=bins_y_tof)[0]
    H_y_tof += np.histogram2d( data['tof']*1e6, data['y'], bins=bins_y_tof)[0]
    
    
    return hv.Image(H_y_tof.T[::-1], bounds=[bins_y_tof[0][0], bins_y_tof[1][0], bins_y_tof[0][-1], bins_y_tof[1][-1]]).opts(xlabel='ToF (µs)', ylabel='pixel (y)', width=900, logz=True, clim=(1e-3, None), cmap="jet")

@callback
def on_data_tof(in_data):
    global tof_buffer
    if in_data['type'] == ChannelDataType.TOF.value:
        '''recieved TOF data'''
        #print(in_data['data'])
        tof_buffer.send(in_data['data']) 
        

In [ ]:
hv.DynamicMap(my_x_tof, streams=[tof_buffer])

In [ ]:
'''Plotting rolling average'''

#centroid_data=None

averaging_window = 5

ring_length = 200

rolling_buffer = deque(maxlen=ring_length)

centroid_df_data = pd.DataFrame({'nr':[], 'x':[], 'y':[], 'tof':[], 'tot_avg':[],\
                                 'tot_max':[], 'clustersize':[],})

rolling_buffer = Pipe(centroid_df_data)


def my_roll_average(data):
    global roll_buffer
    global averaging_window
    global ring_length
    
    for i in data['clustersize'].rolling(averaging_window, center=True).mean()[::averaging_window]:
        if not pd.isnull(i):
            roll_buffer.append(i)
    
    return (hv.Curve(roll_buffer).opts(xlim=(0, len(roll_buffer)))).opts(width=900)


@callback
def on_data_centroid(in_data):
    global rolling_buffer
    global centroid_data
    
    if in_data['type'] == ChannelDataType.CENTROID.value:
        '''recieved centroid data'''

        #centroid_data = in_data['data']['clustersize'].rolling(averaging_window).mean()        

        rolling_buffer.send(in_data['data'])

In [ ]:
hv.DynamicMap(my_roll_average, streams=[rolling_buffer])